# KNN and decision tree classification
In this lab, we will work with the DonorChoose.org dataset from KDD Cup 14 to predict whether a funding request is exciting or not. Keep your work organized as we will use the same data and problem in the following labs as well. 

## Download the datasets from the KDD website
The first step is to download the dataset from [Kaggle website](http://www.kaggle.com/c/kdd-cup-2014-predicting-excitement-at-donors-choose/data). You can look at the definition of 'is exciting' as well as some explanation of the data fields in the given link. In this lab, we will use Python, Pandas, numpy and scikit-learn.

In [162]:
import pandas as pd
import pylab as pl
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

First, check the size of the datasets:

In [163]:
# example:
df_projects  = pd.read_csv('Data/projects.csv')
df_outcomes  = pd.read_csv('Data/outcomes.csv')
#df_projects.shape[0]

The dataset contains
- projects    :  664098  
- resources  :  3667217
- essays       :  664098
- outcomes  :   619326
- donations  :   3097989



## plotting the data
Plot the exciting projects based on their date. See whether there is any interesting result there.

In [164]:
df_projects['date_posted'] = df_projects['date_posted'].apply(pd.to_datetime)

In [165]:
df_combined = pd.merge(df_projects, df_outcomes, on='projectid')

In [166]:
df_combined.head()

projectid                    teacher_acctid  \
0  62526d85d2a1818432d03d600969e99c  ebc7c90b6c92a069432e0714b8d93dfd   
1  33d59ac771b80222ad63ef0f4ac47ade  de83b4c1f6428a15032c207c1d5e572a   
2  1a3aaeffc56dd2a421e37d8298024c0a  f4c9ed095b85458dcf858e25f203af00   
3  33aa19ee4da4c5adf47d0dfb84fab5ef  17768031eb40de8d4497dbb54df48742   
4  e31c0ea8b68f404699dfb0d39e9bc99b  0f1bc5b4700fd33383be104442660178   

                           schoolid  school_ncesid  school_latitude  \
0  5aca9711ff0e4b37db48701f46f73036   1.713710e+11        41.972419   
1  d91a805b213bf74ae77b94e0de2b73ad   1.601530e+11        43.501154   
2  9310d3eb447a4e46bc5fc31ed007ceac   3.302610e+11        42.888244   
3  9ac70da58322783f82152eecc140a812   5.103240e+11        37.476158   
4  cb9f688cf59e3ee22a087d616ca8f5d7   1.709930e+11        41.952851   

   school_longitude  school_city school_state  school_zip school_metro  \
0        -88.174597     Bartlett           IL       60103     suburban   
1       -112.056780  Idaho Falls           ID       83402        urban   
2        -71.320224        Derry           NH        3038     suburban   
3        -77.488397     Richmond           VA       23224        urban   
4        -87.650233      Chicago           IL       60613        urban   

             ...             at_least_1_teacher_referred_donor fully_funded  \
0            ...                                             f            t   
1            ...                                           NaN            f   
2            ...                                             f            f   
3            ...                                             f            f   
4            ...                                             f            t   

  at_least_1_green_donation great_chat  \
0                         f          t   
1                       NaN          f   
2                         t          f   
3                         t          f   
4                         t          f   

  three_or_more_non_teacher_referred_donors  \
0                                         t   
1                                       NaN   
2                                         f   
3                                         f   
4                                         f   

  one_non_teacher_referred_donor_giving_100_plus  \
0                                              t   
1                                            NaN   
2                                              f   
3                                              f   
4                                              t   

  donation_from_thoughtful_donor great_messages_proportion  \
0                              f                        80   
1                            NaN                       NaN   
2                              f                       NaN   
3                              f                       NaN   
4                              f                        50   

  teacher_referred_count non_teacher_referred_count  
0                      0                          6  
1                    NaN                        NaN  
2                      0                          2  
3                      0                          1  
4                      0                          2  

[5 rows x 46 columns]

In [167]:
df_exciting = df_combined[df_combined['is_exciting'] == 't']
df_unsorted = df_exciting[['schoolid']].groupby(df_exciting['date_posted']).count().reset_index()
df_unsorted = df_unsorted.rename(columns = {'schoolid':'Count of exciting projects'})
df_unsorted.plot(x='date_posted', y='Count of exciting projects')

You will see that there is no exciting projects before 2010-04-14. This can effect the accuracy of prediction. The reason is DonorsChoose.org simply did not keep track of some of the requirements needed to decide 'is_exciting'. How can you avoid this issue in your prediction?

I can avoid this issue in my prediction by running the test data from 2010-04-14 to 2014-01-01. Alternatively, if I want to be more thorough, I can cluster the previous projects for which exciting features are not tracked through cluster analysis. 

## Load the CSV files into Pandas and preprocess as needed
Let us stick to projects.csv and outcomes.csv for the tasks in this lab session for now. Check whether there are Null instances or other problems in your data  you should take care of.

In [226]:
#df_projects.isnull().values.any()
#df_outcomes.isnull().values.any()

#Given that this is a time series data, it makes sense to fill in the entry from the previous record.
df_combined = df_combined.fillna(method='bfill')
#df_outcomes = df_outcomes.fillna(method='bfill')

## Prediction tasks
In this lab we ask you to work on the following two prediction tasks. The first one is a binary classfication and the second one is a multi-class classification problem.

### Task 1
Predict whether a project is exciting by picking the features you chose from the dataset. Feel free to make new features from the existing ones.

### Task 2
Predict the Poverty level of the school. Which feature would you pick from the dataset?

## Choose your features
For each task, choose some of the data fields from projects.csv that you think could be used as useful features for predicting the 'is-exciting' and poverty level outcome.

In [227]:
df_projects.columns

Index(['projectid', 'teacher_acctid', 'schoolid', 'school_ncesid',
       'school_latitude', 'school_longitude', 'school_city', 'school_state',
       'school_zip', 'school_metro', 'school_district', 'school_county',
       'school_charter', 'school_magnet', 'school_year_round', 'school_nlns',
       'school_kipp', 'school_charter_ready_promise', 'teacher_prefix',
       'teacher_teach_for_america', 'teacher_ny_teaching_fellow',
       'primary_focus_subject', 'primary_focus_area',
       'secondary_focus_subject', 'secondary_focus_area', 'resource_type',
       'poverty_level', 'grade_level', 'fulfillment_labor_materials',
       'total_price_excluding_optional_support',
       'total_price_including_optional_support', 'students_reached',
       'eligible_double_your_impact_match', 'eligible_almost_home_match',
       'date_posted'],
      dtype='object')

In [234]:
df_projects_sample = df_combined[['total_price_excluding_optional_support', 'total_price_including_optional_support','students_reached']]
#df_projects_sample = df_combined[['school_metro', 'school_county', 'grade_level', 'primary_focus_area', 'students_reached', 'poverty_level']]

## Form your train and test set
Separate a portion of your data as train and the other portion as test set to work with. Keep these two sets fixed for all of your experiments and choose them randomly. You can use the following:

In [236]:
from sklearn.cross_validation import train_test_split

In [237]:
# same size for both data frames
project_train, project_test, outcomes_train, outcomes_test = train_test_split(df_projects_sample, df_combined.is_exciting, test_size=0.3, random_state=42)

## Train your models and test on your test data

### First step

Train KNN and Decision Tree models with your training set for each task. Choose some initial parameters for your models and see how they perform on the test set for these tasks.

In [256]:
# You may need to import these:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree

# for KNN

In [239]:
#le = preprocessing.LabelEncoder()
#le.fit(project_train[['school_metro'])
neighbor = KNeighborsClassifier(n_neighbors=3)
neighbor.fit(project_train, outcomes_train) 

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_neighbors=3, p=2, weights='uniform')

In [243]:
neighbor.predict(project_test)

array(['f', 'f', 'f', ..., 'f', 'f', 'f'], dtype=object)

In [255]:
neighbor.score(project_test, outcomes_test)

0.92844379379756514

In [259]:
neighbor.predict_proba(project_test)

array([[ 0.66666667,  0.33333333],
       [ 0.66666667,  0.33333333],
       [ 1.        ,  0.        ],
       ..., 
       [ 0.66666667,  0.33333333],
       [ 1.        ,  0.        ],
       [ 1.        ,  0.        ]])

In [271]:
project_test_mini = project_test[:5]

## Explore the models you just built

### kNN
For kNN, take a few points and retrieve neirest neighbors for each of them? Do the neighbors make sense? 
Would you change the distance function to get "better" neighbors?

In [266]:
neighbor.kneighbors(X=project_test_mini, n_neighbors=4, return_distance=True)

(array([[ 0.01414214,  0.19209373,  0.72346389,  1.14978259],
        [ 0.15620499,  0.17029386,  0.18439089,  0.27658633],
        [ 0.01414214,  0.32649655,  0.34058773,  0.94366308],
        [ 1.1088733 ,  1.12004464,  1.12004464,  1.12004464],
        [ 1.86772589,  2.16333077,  2.32894826,  2.48419001]]),
 array([[168652, 392563, 346462, 308242],
        [ 55342, 260113, 355423, 228927],
        [ 67483,  42283, 218212,  76313],
        [ 69311,  26924,  63803, 120299],
        [ 71591, 241842, 144120, 158622]]))

They don't make a lot of sense for the first record. Yes, I'd like to use a different function to get better neighbors.

### Decision Trees
For Decision Trees, can you visualize or print out the tree? Try using sklearn.tree.export_graphviz()

## How do these models change as you vary the parameters?

### For kNN, see what happens when:
- you do or do not scale/normalize features
- you vary k from very small to very large
- you vary distance metrics
- you vary distance weighting

varying k from very small to very large

In [268]:
neighbor = KNeighborsClassifier(n_neighbors=1)
neighbor.fit(project_train, outcomes_train) 
neighbor.score(project_test, outcomes_test)

0.89326042260950067

In [269]:
neighbor = KNeighborsClassifier(n_neighbors=10)
neighbor.fit(project_train, outcomes_train) 
neighbor.score(project_test, outcomes_test)

0.94080668252618438

The prediction accuracy increases. This indicates that the data may be more evenly spread.

varying distance weighting

In [270]:
neighbor = KNeighborsClassifier(n_neighbors=3, weights = 'distance')
neighbor.fit(project_train, outcomes_train) 
neighbor.score(project_test, outcomes_test)

0.91102164716520095

accuracy rates = 0.911 (distance weighting) versus 0.928 (uniform weighting)

In [ ]:
varying distance metrics

In [272]:
neighbor = KNeighborsClassifier(n_neighbors=3, p = 1)
neighbor.fit(project_train, outcomes_train) 
neighbor.score(project_test, outcomes_test)

0.92852990882571396

accuracy rates = 0.9284 (Manhattan distance metric) versus 0.9285 (Euclidean distance metric)

### For decision trees, try similar ideas:
- vary pruning and stopping strategies, use the following parameters:
    - min_impurity_split
    - min_samples_split
- vary splitting criteria, use the following parameters:
    - max_depth
    - mean_samples_leaf
    - max_leaf_nodes
    
Check the reference below for description of these parameters

## Operationalizing this search through the parameter space

Can you put all all of these variations in a nested for loop and run all of them and look at the performance of the models?

## References
http://scikit-learn.org/stable/auto_examples/neighbors/plot_classification.html#sphx-glr-auto-examples-neighbors-plot-classification-py

http://scikit-learn.org/stable/modules/tree.html

Different parameters for decision tree:
http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier